In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [19]:
movieDf = pd.read_csv('movies.csv')
ratingDf = pd.read_csv('ratings.csv')

In [3]:
movieDf.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
ratingDf.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
movieDf.drop(['genres'], axis = 1, inplace = True)

In [11]:
movieDf.head(3)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)


In [21]:
user_item_matrix = ratingDf.pivot(index= 'movieId', columns='userId', values='rating')

In [14]:
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
user_item_matrix.fillna(0, inplace=True)

In [46]:
user_item_matrix.head()

userId,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,3.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [18]:
user_item_matrix.shape


(9724, 610)

In [26]:
user_votes = ratingDf.groupby('userId')['rating'].agg('count')

In [28]:
movies_votes = ratingDf.groupby('movieId')['rating'].agg('count')

In [40]:
user_mask = user_votes[user_votes > 50].index
movie_mask = movies_votes[movies_votes > 10].index

In [41]:
user_item_matrix = user_item_matrix.loc[movie_mask, :]
user_item_matrix = user_item_matrix.loc[:, user_mask]

In [47]:
user_item_matrix.head()


userId,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,3.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [48]:
user_item_matrix.head()

userId,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,3.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [49]:
csr_data = csr_matrix(user_item_matrix.values)

In [50]:
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()

In [51]:
user_item_matrix.head()

,movieId,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [52]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [53]:
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [55]:
recommendations = 10
search_word = 'Jumanji'

In [56]:
movie_search = movieDf[movieDf['title'].str.contains(search_word)]

In [57]:
movie_search

,movieId,title,genres
1,2,Jumanji (1995),Adventure|Children|Fantasy
9636,179401,Jumanji: Welcome to the Jungle (2017),Action|Adventure|Children


In [58]:
#возьмем вторую часть фильма

In [91]:
movie_id = movie_search.iloc[0]['movieId']

In [92]:
movie_id

2

In [93]:
movie_id = user_item_matrix[user_item_matrix['movieId'] == movie_id].index[0]

In [94]:
movie_id

1

In [95]:
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors= recommendations + 1)

In [96]:
indices

array([[  1, 173, 175, 227, 217, 262, 268, 260, 253, 149,  72]],
      dtype=int64)

In [97]:
distances

array([[0.        , 0.37701322, 0.41374278, 0.41616445, 0.42056321,
        0.42542803, 0.43500669, 0.44343178, 0.46261167, 0.46965353,
        0.47425265]])

In [103]:
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

In [105]:
indices_distances = list(zip(indcces_list, distances_list))

In [106]:
indices_distances

[(1, 0.0),
 (173, 0.3770132181136182),
 (175, 0.4137427842741309),
 (227, 0.41616445019173887),
 (217, 0.4205632052050985),
 (262, 0.4254280334192815),
 (268, 0.43500669079100873),
 (260, 0.443431779832277),
 (253, 0.46261166941937715),
 (149, 0.46965352842617836),
 (72, 0.4742526487742069)]

In [107]:
indices_distances_sort = sorted(indices_distances, key = lambda x: x[1], reverse=True)

In [109]:
indices_distances_sort = indices_distances_sort[: -1]

In [110]:
indices_distances_sort

[(72, 0.4742526487742069),
 (149, 0.46965352842617836),
 (253, 0.46261166941937715),
 (260, 0.443431779832277),
 (268, 0.43500669079100873),
 (262, 0.4254280334192815),
 (217, 0.4205632052050985),
 (227, 0.41616445019173887),
 (175, 0.4137427842741309),
 (173, 0.3770132181136182)]

In [114]:
recom_list = []
for ind_dist in indices_distances_sort:
    movie_id = user_item_matrix.iloc[ind_dist[0]]['movieId']
    id = movieDf[movieDf['movieId'] == movie_id].index
    title = movieDf.iloc[id]['title'].values[0]
    dist = ind_dist[1]
    recom_list.append({'Title' : title, 'Distance': dist})

In [115]:
recom_list

[{'Title': 'Casper (1995)', 'Distance': 0.4742526487742069},
 {'Title': 'Stargate (1994)', 'Distance': 0.46965352842617836},
 {'Title': 'Nightmare Before Christmas, The (1993)',
  'Distance': 0.46261166941937715},
 {'Title': 'Home Alone (1990)', 'Distance': 0.443431779832277},
 {'Title': 'Beauty and the Beast (1991)', 'Distance': 0.43500669079100873},
 {'Title': 'Aladdin (1992)', 'Distance': 0.4254280334192815},
 {'Title': 'Jurassic Park (1993)', 'Distance': 0.4205632052050985},
 {'Title': 'Mrs. Doubtfire (1993)', 'Distance': 0.41616445019173887},
 {'Title': 'Mask, The (1994)', 'Distance': 0.4137427842741309},
 {'Title': 'Lion King, The (1994)', 'Distance': 0.3770132181136182}]

In [116]:
#Для рекомендательной системы мы использовали фильм "Джуманджи"
#судя по полученному списку, можно сделать вывод об успешной разработке рекомендательной системы
#так как в списке много фильмов схожи с исходным
#например Парк Юрского периода, Король Лев, Маска имеют общие черты с фильмом Джуманджи